# Deep Learning - Exercise 1

## Covertype Data Set

Data set that was chosen for this exercise is the Covertype Data Set and the problem is a multiclass classification problem - predicting forest cover type from cartographic variables only (no remotely sensed data).

[Link to dataset in UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Covertype)

### Imports

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

### Read the data

In [2]:
df = pd.read_csv('covtype.data', names = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 
                                          'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 
                                          'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 
                                          'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1', 
                                          'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 
                                          'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 
                                          'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 
                                          'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 
                                          'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 
                                          'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 
                                          'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 
                                          'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 
                                          'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 
                                          'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 
                                          'Soil_Type40', 'Cover_Type'])
label_column = 'Cover_Type'

X = df.loc[:, df.columns != label_column].astype(float)
y = df.loc[:, label_column]

### Prepare the data as input for the network

In [3]:
# Normalize the data
X_values = X.iloc[:, :10].values.reshape(-1, 10) 
min_max_scaler = MinMaxScaler()
X.iloc[:, :10] = min_max_scaler.fit_transform(X_values)

### Split the data into train, test and validation

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

### Hyperparameter tuning

In [5]:
epochs = 1000
learning_rate = 0.01
keep_probability = 0.85
max_norm_constraint = 3
regularization = 0.1
hidden_layer_size = 256
batch_size = 100

### Build the network

In [6]:
# Reset
tf.reset_default_graph()

# Placeholders
x = tf.placeholder(tf.float32, [None, X_train.shape[1]], 'x')
y = tf.placeholder(tf.float32, [None], 'y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

# Regularization
#regularizer = tf.contrib.layers.l2_regularizer(scale=regularization)

# Model
model = tf.layers.dense(x, hidden_layer_size, tf.nn.relu)#, kernel_initializer=regularizer)
model = tf.nn.dropout(model, keep_prob)
model = tf.layers.dense(model, y_train.nunique())#, kernel_initializer=regularizer)
logits = tf.nn.softmax(model)

# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
#cost = tf.reduce_mean(tf.squared_difference(logits, y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.cast(tf.argmax(logits, 1), tf.float32), y)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

### Training the network

In [7]:
def get_batch(X, y, current_batch_index):
    batch_start_index = current_batch_index * batch_size
    batch_end_index = (current_batch_index + 1) * batch_size
    batch_features = X.iloc[batch_start_index:batch_end_index].values
    batch_labels = y.iloc[batch_start_index:batch_end_index].values
    return batch_features, batch_labels

def print_stats(session, feature_batch, label_batch, cost, accuracy):
    feed_cost = {x: feature_batch, y: label_batch, keep_prob: 1.0 }
    feed_valid = {x: X_validate, y: y_validate, keep_prob: 1.0 }
    cost = session.run(cost, feed_cost)
    accuracy = session.run(accuracy, feed_valid)
    print "cost: %.2f" % cost, "accuracy: %.2f" % accuracy

In [8]:
save_model_path = './saver'
print 'Training...'
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        n_batches = X_train.shape[0] / batch_size
        for batch_i in range(n_batches):
            batch_features, batch_labels = get_batch(X_train, y_train, batch_i)
            sess.run(optimizer, feed_dict={x: batch_features, y: batch_labels, keep_prob: keep_probability})
            print 'Epoch {:>2}, Batch {}: '.format(epoch + 1, batch_i)
            print_stats(sess, batch_features, batch_labels, cost, accuracy)
    tf.train.Saver().save(sess, save_model_path)

Training...


InvalidArgumentError: logits and labels must be same size: logits_size=[100,7] labels_size=[1,100]
	 [[Node: SoftmaxCrossEntropyWithLogits = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](Reshape, Reshape_1)]]

Caused by op u'SoftmaxCrossEntropyWithLogits', defined at:
  File "/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/anaconda2/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    s

### Test the network

In [ ]:
print 'Testing...'
with tf.Session() as sess:
    test_batch_acc_total = 0
    test_batch_count = 0
    n_batches = X_test.shape[0] / batch_size
    for batch_i in range(n_batches):
        batch_features, batch_labels = get_batch(X_test, y_test, batch_i)
        test_batch_acc_total += sess.run(accuracy, feed_dict={x: batch_features, y: batch_labels, keep_prob: 1.0})
        test_batch_count += 1
    print 'Testing Accuracy: {}\n'.format(test_batch_acc_total / test_batch_count)